<a href="https://colab.research.google.com/github/OgunSerifOnargan/arkitek_notebooks/blob/main/obj_trcking_with_yt_dlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

import os
HOME = os.getcwd()
print(HOME)

# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt
!pip install yt_dlp
!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/arkitek_fobi_analiz/useful_codes/ultralytics/plotting.py','/usr/local/lib/python3.10/dist-packages/ultralytics/utils/')

In [ ]:
@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 60
    match_thresh: float = 0.80
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False



!pip install supervision==0.1.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)


from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

from typing import List
import yt_dlp
import numpy as np
import unicodedata

#rename files function work with a list
"""
old_char_list: değiştirilmesi istenen karakter listesi alır
new_char: string değeri alır

old_char_list listesinin tüm elemanlarını dosya ismi ve eklentisini ayırır ve dosya isminde varsa new_char ile değiştirir
"""
def rename_file_replace_char(file_path, old_char_list, new_char):
    filename = os.path.basename(file_path)
    for old_char in old_char_list:
        filename = filename.replace(old_char, new_char)

    new_file_path = os.path.join(os.path.dirname(file_path), filename)
    os.rename(file_path, new_file_path)



#download a youtube playlist function

def download_playlist(playlist_url, output_folder):
    ydl_opts = {
        'format': 'best',
        'quiet': False,
        'outtmpl': os.path.join(output_folder, '%(playlist_index)s - %(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([playlist_url])
        except:
             pass

def download_video(video_url, output_folder):
    ydl_opts = {
        'format': 'best',
        'quiet': False,
        'outtmpl': os.path.join(output_folder, '%(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([video_url])
            print("Video indirme tamamlandı.")
        except Exception as e:
            print(f'Hata oluştu: {str(e)}')




# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids



supervision.__version__: 0.1.0


In [ ]:
# !pip install pytube
from IPython import display
display.clear_output()

In [ ]:
# from pytube import YouTube

# # Replace 'YOUR_VIDEO_URL' with the URL of the YouTube video you want to download
# video_url = 'https://youtu.be/OEoTLl2Kgvo?si=8EYBUqxW-BBKdwKi'

# # Create a YouTube object
# yt = YouTube(video_url)

# # Get the stream with the highest resolution
# highest_resolution_stream = yt.streams.get_highest_resolution()

# # Print information about the selected stream (optional)
# print("Video Title:", yt.title)
# print("Resolution:", highest_resolution_stream.resolution)
# print("File Size:", highest_resolution_stream.filesize)

# # Set the path where you want to save the video
# # Replace 'YOUR_DOWNLOAD_PATH' with your desired file path

# # Download the video to the specified path
# highest_resolution_stream.download(output_path="/content/")

# print("Video downloaded successfully!")
# video_path = "/content/" + yt.title + ".mp4"
# SOURCE_VIDEO_PATH = video_path
# print(SOURCE_VIDEO_PATH)
# HOME = "/content"

In [ ]:
HOME = "/content/"
videos_path = "/content/videos/"
if not os.path.exists(videos_path):
        os.makedirs(videos_path)


# **VIDEO DOWNLOADER**

In [ ]:
# @title
video_url  = r'https://www.youtube.com/watch?v=59fsRDsJh48'
download_video(video_url,videos_path)
display.clear_output()
print('Video indirme tamamlandı.')

Video indirme tamamlandı.


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8x.pt')
model.fuse()

# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
print(CLASS_NAMES_DICT)
# class_ids of interest - car, motorcycle, bus and truck
CLASS_ID = [14,15,16]

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients


{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [ ]:
@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 60
    match_thresh: float = 0.80
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False


In [ ]:
from tqdm.notebook import tqdm
import gc
for root,dirs,files in os.walk(videos_path):
    for file in files:
      SOURCE_VIDEO_PATH = os.path.join(root,file)
      file_name = file.split('/')[0]
      print(file_name)
      TARGET_VIDEO_PATH = f"/content/drive/MyDrive/{file_name}.mp4"
      # create BYTETracker instance
      byte_tracker = BYTETracker(BYTETrackerArgs())
      # create VideoInfo instance
      video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
      # create frame generator
      generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
      # create LineCounter instance
      #line_counter = LineCounter(start=LINE_START, end=LINE_END)
      # create instance of BoxAnnotator and LineCounterAnnotator
      box_annotator = BoxAnnotator(color=ColorPalette(), thickness=2, text_thickness=2, text_scale=1)
      line_annotator = LineCounterAnnotator(thickness=4, text_thickness=4, text_scale=2)

      # open target video file
      with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
          # loop over video frames
          for frame in tqdm(generator, total=video_info.total_frames):
              # model prediction on single frame and conversion to supervision Detections
              results = model(frame,verbose = False)
              detections = Detections(
                  xyxy=results[0].boxes.xyxy.cpu().numpy(),
                  confidence=results[0].boxes.conf.cpu().numpy(),
                  class_id=results[0].boxes.cls.cpu().numpy().astype(int)
              )
              # filtering out detections with unwanted classes
              mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
              detections.filter(mask=mask, inplace=True)
              # tracking detections
              tracks = byte_tracker.update(
                  output_results=detections2boxes(detections=detections),
                  img_info=frame.shape,
                  img_size=frame.shape
              )
              tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
              detections.tracker_id = np.array(tracker_id)
              # filtering out detections without trackers
              mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
              detections.filter(mask=mask, inplace=True)
              # format custom labels
              labels = [
                  f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
                  for _, confidence, class_id, tracker_id
                  in detections
              ]
              # updating line counter
      #        line_counter.update(detections=detections)
              # annotate and display frame
              frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
      #        line_annotator.annotate(frame=frame, line_counter=line_counter)
              sink.write_frame(frame)
          gc.collect()

High Anxiety (1977) - The Birds [HD].mp4


  0%|          | 0/3697 [00:00<?, ?it/s]

## 🏆 Congratulations

### Learning Resources

Roboflow has produced many resources that you may find interesting as you advance your knowledge of computer vision:

- [Roboflow Notebooks](https://github.com/roboflow/notebooks): A repository of over 20 notebooks that walk through how to train custom models with a range of model types, from YOLOv7 to SegFormer.
- [Roboflow YouTube](https://www.youtube.com/c/Roboflow): Our library of videos featuring deep dives into the latest in computer vision, detailed tutorials that accompany our notebooks, and more.
- [Roboflow Discuss](https://discuss.roboflow.com/): Have a question about how to do something on Roboflow? Ask your question on our discussion forum.
- [Roboflow Models](https://roboflow.com): Learn about state-of-the-art models and their performance. Find links and tutorials to guide your learning.

### Convert data formats

Roboflow provides free utilities to convert data between dozens of popular computer vision formats. Check out [Roboflow Formats](https://roboflow.com/formats) to find tutorials on how to convert data between formats in a few clicks.

### Connect computer vision to your project logic

[Roboflow Templates](https://roboflow.com/templates) is a public gallery of code snippets that you can use to connect computer vision to your project logic. Code snippets range from sending emails after inference to measuring object distance between detections.